In [1]:
import sqlite3
import pandas as pd
import os
import numpy as np
import warnings

rootpath = "C:\\Users\\Dave\\Google Drive\\College\\FE550\\Assignment3"
datapath = os.path.join(rootpath,"Data")

print(datapath)


C:\Users\Dave\Google Drive\College\FE550\Assignment3\Data


In [2]:
# Export Json file to be used in presentation
def JExport(f,df):
    outpath = os.path.join(datapath,f)
    df.to_json(outpath,orient = 'index',date_format = 'epoch', double_precision = 10, force_ascii = True, date_unit = 'ms', default_handler = None)
    


In [3]:

dbpath = os.path.join(datapath,"database.sqlite")
conn = sqlite3.connect(dbpath)

#Getting average scores across the board
SQLplayerAttr = "select pa1.player_api_id, p.player_name, avg(overall_rating) overall_rating, \
                    avg(crossing) crossing, avg(finishing) finishing, avg(short_passing) short_passing, \
                    avg(volleys) volleys, avg(dribbling) dribbling, avg(long_passing) long_passing, \
                    avg(ball_control) ball_control, avg(POS) Posnum, \
                    case when avg(POS) >=10 then 'FWD' \
                        when avg(POS) >= 6 then 'MID' \
                        when avg(POS) >= 2 then 'DEF' \
                        when avg(POS) = 1 then 'GK' \
                        else null \
                    end Position\
                from player_attributes pa1 inner join \
                     player p on p.player_api_id = pa1.player_api_id \
                    inner join \
                        (select home_player_1 as PID, home_player_Y1 as POS from match union all \
                        select home_player_2, home_player_Y2 from match union all \
                        select home_player_3, home_player_Y3 from match union all \
                        select home_player_4, home_player_Y4 from match union all \
                        select home_player_5, home_player_Y5 from match union all \
                        select home_player_6, home_player_Y6 from match union all \
                        select home_player_7, home_player_Y7 from match union all \
                        select home_player_8, home_player_Y8 from match union all \
                        select home_player_9, home_player_Y9 from match union all \
                        select home_player_10, home_player_Y10 from match union all\
                        select home_player_11, home_player_Y11 from match) tblpos \
                        on tblpos.PID = pa1.player_api_id \
                     group by pa1.player_api_id, player_name"


#print(SQLplayerAttr)

plyrData = pd.read_sql_query(SQLplayerAttr,conn)
plyrData.head()


,player_api_id,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position
0,2625,"Patryk Rachwal,18",60.142857,50.142857,47.285714,56.500000,38.000000,54.142857,65.571429,59.142857,6.5,MID
1,2752,Diego Mainz,69.380952,36.428571,37.428571,54.809524,20.428571,44.857143,59.571429,55.571429,3.0,DEF
2,2768,Jose Dorado,69.285714,42.238095,42.761905,64.714286,28.761905,40.238095,57.000000,56.380952,3.0,DEF
3,2770,Ignacio Gonzalez,71.133333,61.866667,67.666667,72.533333,67.133333,75.066667,70.266667,76.733333,7.6,MID
4,2790,Alberto Rey,70.200000,70.000000,37.400000,68.000000,43.000000,60.800000,69.800000,66.800000,3.0,DEF


In [4]:
plyrData.count()

player_api_id     10543
player_name       10543
overall_rating    10543
crossing          10543
finishing         10543
short_passing     10543
volleys           10101
dribbling         10543
long_passing      10543
ball_control      10543
Posnum            10529
Position          10510
dtype: int64

In [5]:
# Remove any players that do not have the attributes we are looking at
# This is the base data we are using for players
cleanplyrData = plyrData.dropna()
cleanplyrData = cleanplyrData.set_index('player_api_id')
cleanplyrData.index.name=None
cleanplyrData['player_api_id'] = cleanplyrData.index
cleanplyrData.count()

player_name       10072
overall_rating    10072
crossing          10072
finishing         10072
short_passing     10072
volleys           10072
dribbling         10072
long_passing      10072
ball_control      10072
Posnum            10072
Position          10072
player_api_id     10072
dtype: int64

In [6]:
cleanplyrData.head()


,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position,player_api_id
2625,"Patryk Rachwal,18",60.142857,50.142857,47.285714,56.500000,38.000000,54.142857,65.571429,59.142857,6.5,MID,2625
2752,Diego Mainz,69.380952,36.428571,37.428571,54.809524,20.428571,44.857143,59.571429,55.571429,3.0,DEF,2752
2768,Jose Dorado,69.285714,42.238095,42.761905,64.714286,28.761905,40.238095,57.000000,56.380952,3.0,DEF,2768
2770,Ignacio Gonzalez,71.133333,61.866667,67.666667,72.533333,67.133333,75.066667,70.266667,76.733333,7.6,MID,2770
2790,Alberto Rey,70.200000,70.000000,37.400000,68.000000,43.000000,60.800000,69.800000,66.800000,3.0,DEF,2790


In [7]:
# Build JSON for player attributes by decile
cleanplyrData['decile'] = pd.qcut(cleanplyrData['overall_rating'],10,labels=False)

In [8]:
def get_avg_byDecile(dfdecile, decColname):
    # we care about crossing, finishing, short_passing, volleys, dribbling, long_passing, ball_control
    attrary = ['overall_rating','crossing', 'finishing', 'short_passing', 'volleys', 'dribbling', 'long_passing', 'ball_control'];
    outary = {};
    outary[decColname] = dfdecile[decColname].unique().item()+1;
    for a in attrary:
        tmpm = dfdecile[a].mean();
        outary[a] = tmpm;
    return outary;

In [28]:
def get_norm_byDecile(dfdecile, decColname):
    # we care about crossing, finishing, short_passing, volleys, dribbling, long_passing, ball_control
    attrary = ['overall_rating','crossing', 'finishing', 'short_passing', 'volleys', 'dribbling', 'long_passing', 'ball_control'];
    outary = {};
    outary[decColname] = dfdecile[decColname].unique().item()+1;
    for a in attrary:
        tmpm = (dfdecile[a] - dfdecile[a].min())/(dfdecile[a].max() - dfdecile[a].min());
        newvar = 'norm_' + a;
        outary[newvar] = tmpm.mean();
    return outary;

In [24]:
# Player JSON needs to be the avg score by decile for each attribute
lstPlyr = [];

for i in list(range(0, 10)):
    lstPlyr.append(get_avg_byDecile(cleanplyrData[cleanplyrData['decile']==i],'decile'));


dfPlyr = pd.DataFrame(lstPlyr);
dfPlyr = dfPlyr.reset_index();
#dfPlyr = dfPlyr.set_index('decile');
#dfPlyr.index.name=None;
#dfPlyr['decile'] = dfPlyr.index;

print(dfPlyr.head(5));
print(dfPlyr.count());

   index  ball_control   crossing  decile  dribbling  finishing  long_passing  \
0      0     50.548251  43.357959       1  48.242473  37.865842     45.873155   
1      1     55.538518  47.632721       2  52.285061  42.783065     49.331735   
2      2     57.335015  48.971963       3  53.361062  44.112593     51.430834   
3      3     59.329165  51.302241       4  55.010807  45.046851     53.369594   
4      4     61.505478  53.316466       5  57.290653  47.617790     54.903559   

   overall_rating  short_passing    volleys  
0       56.315304      51.031493  36.654876  
1       61.059023      54.961771  40.957889  
2       63.193539      56.489010  42.330591  
3       64.791238      58.859327  44.376128  
4       66.302621      60.367426  46.600168  
index             10
ball_control      10
crossing          10
decile            10
dribbling         10
finishing         10
long_passing      10
overall_rating    10
short_passing     10
volleys           10
dtype: int64


In [19]:
JExport('plyrattr.json',dfPlyr);

In [29]:
# Build a normalized data view so we can compare the different deciles for importance
lstNorm = [];

for i in list(range(0, 10)):
    lstNorm.append(get_norm_byDecile(cleanplyrData[cleanplyrData['decile']==i],'decile'));


dfPlyrNorm = pd.DataFrame(lstNorm);
dfPlyrNorm = dfPlyrNorm.reset_index();
#dfPlyr = dfPlyr.set_index('decile');
#dfPlyr.index.name=None;
#dfPlyr['decile'] = dfPlyr.index;

print(dfPlyrNorm.head(5));
print(dfPlyrNorm.count());

   index  decile  norm_ball_control  norm_crossing  norm_dribbling  \
0      0       1           0.666824       0.600939        0.621213   
1      1       2           0.713770       0.660837        0.664979   
2      2       3           0.731210       0.610275        0.674549   
3      3       4           0.707092       0.633591        0.657297   
4      4       5           0.765235       0.631321        0.699654   

   norm_finishing  norm_long_passing  norm_overall_rating  norm_short_passing  \
0        0.517303           0.547413             0.795854            0.597699   
1        0.568810           0.596793             0.557988            0.669682   
2        0.555014           0.637400             0.537276            0.696004   
3        0.564033           0.656009             0.505837            0.712053   
4        0.608102           0.631726             0.527582            0.720691   

   norm_volleys  
0      0.512138  
1      0.546720  
2      0.521854  
3      0.498348  
4 

In [30]:
JExport('plyrattrnorm.json',dfPlyrNorm);

In [11]:
# get the top 100 players  using min for duplicate ranks
cleanplyrData['overallrank'] = cleanplyrData['overall_rating'].rank(ascending=False, method='dense')
cleanplyrData.head()

,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position,player_api_id,decile,overallrank
2625,"Patryk Rachwal,18",60.142857,50.142857,47.285714,56.500000,38.000000,54.142857,65.571429,59.142857,6.5,MID,2625,1,3602.0
2752,Diego Mainz,69.380952,36.428571,37.428571,54.809524,20.428571,44.857143,59.571429,55.571429,3.0,DEF,2752,6,1909.0
2768,Jose Dorado,69.285714,42.238095,42.761905,64.714286,28.761905,40.238095,57.000000,56.380952,3.0,DEF,2768,6,1932.0
2770,Ignacio Gonzalez,71.133333,61.866667,67.666667,72.533333,67.133333,75.066667,70.266667,76.733333,7.6,MID,2770,7,1508.0
2790,Alberto Rey,70.200000,70.000000,37.400000,68.000000,43.000000,60.800000,69.800000,66.800000,3.0,DEF,2790,6,1722.0


In [12]:
df_topplayers = cleanplyrData[cleanplyrData['overallrank']<=100]
df_topplayers.head()

,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position,player_api_id,decile,overallrank
18925,Ezequiel Lavezzi,82.193548,77.419355,75.967742,76.516129,77.032258,86.096774,59.709677,84.483871,9.615385,MID,18925,9,78.0
19533,Neymar,83.800000,70.320000,83.480000,75.680000,81.240000,91.600000,72.200000,90.040000,10.000000,FWD,19533,9,40.0
20276,Hulk,82.571429,77.642857,83.464286,78.392857,72.571429,83.357143,73.321429,82.678571,9.926829,MID,20276,9,68.0
22543,Radamel Falcao,83.851852,58.259259,87.740741,67.962963,88.777778,77.740741,55.333333,80.555556,10.337838,FWD,22543,9,38.0
25759,Gonzalo Higuain,83.444444,70.138889,88.500000,70.555556,82.388889,80.611111,58.027778,82.305556,10.382353,FWD,25759,9,49.0


In [145]:
JExport('top100players.json',df_topplayers);

In [31]:
# Player JSON needs to be the avg score by decile for each attribute, including the position this time
# for this we are going to add a decile by position
posary = ['FWD','MID','DEF'];

for p in posary:
    cleanplyrData.loc[(cleanplyrData['Position']==p),'posdecile'] = pd.qcut(cleanplyrData[(cleanplyrData['Position']==p)]['overall_rating'],10,labels=False)
    
    
cleanplyrData[(cleanplyrData['Position']=='FWD')].head()

,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position,player_api_id,decile,posdecile
2983,Jorge Molina,72.050000,43.700000,76.000000,60.050000,76.900000,61.350000,50.000000,70.400000,10.421053,FWD,2983,7,7.0
3377,Luca Belingheri,66.515152,66.303030,57.696970,69.090909,59.393939,66.333333,64.060606,67.727273,10.000000,FWD,3377,4,4.0
3517,Alexandr Kerzhakov,78.857143,72.238095,78.619048,76.000000,68.952381,72.857143,68.142857,79.238095,10.777778,FWD,3517,9,9.0
4999,Sebastian Olszar,57.250000,40.000000,61.500000,50.000000,52.000000,49.000000,36.000000,63.000000,11.000000,FWD,4999,0,0.0
5223,Raffaele Rubino,61.636364,49.181818,60.363636,55.000000,49.909091,59.181818,45.000000,62.000000,10.000000,FWD,5223,1,1.0


In [32]:
cleanplyrData[(cleanplyrData['posdecile']!=cleanplyrData['decile'])].head()

,player_name,overall_rating,crossing,finishing,short_passing,volleys,dribbling,long_passing,ball_control,Posnum,Position,player_api_id,decile,posdecile
2790,Alberto Rey,70.200000,70.000000,37.400000,68.000000,43.000000,60.800000,69.800000,66.800000,3.000000,DEF,2790,6,7.0
2973,Ugur Inceman,68.777778,64.000000,55.166667,73.555556,57.444444,59.888889,71.333333,72.555556,6.142857,MID,2973,6,5.0
2984,Sergio Aragones,68.500000,23.500000,20.500000,30.125000,12.250000,22.500000,48.000000,31.500000,1.000000,GK,2984,5,NaN
3140,Alexandre Hauw,65.857143,57.000000,53.857143,69.000000,54.000000,60.428571,67.142857,66.428571,6.857143,MID,3140,4,3.0
3316,Oscar Sanchez,66.769231,72.692308,53.384615,62.846154,42.230769,61.384615,67.384615,62.000000,3.000000,DEF,3316,4,5.0


In [33]:
#clean up the data a little, we don't need GK in the data
df_posdata = cleanplyrData[cleanplyrData.Position.isin(posary)]
df_posdata.count()

player_name       9227
overall_rating    9227
crossing          9227
finishing         9227
short_passing     9227
volleys           9227
dribbling         9227
long_passing      9227
ball_control      9227
Posnum            9227
Position          9227
player_api_id     9227
decile            9227
posdecile         9227
dtype: int64

In [34]:
lstposPlyr = [];
tmpposary = [];

for p in posary:
    for i in list(range(0, 10)):
        tmpposary = get_norm_byDecile(df_posdata[(df_posdata['Position']==p) & (df_posdata['posdecile']==i)],'posdecile');
        tmpposary['position'] = p;
        lstposPlyr.append(tmpposary);
                   


dfPosPlyrdecile = pd.DataFrame(lstposPlyr);

dfPosPlyrdecile.head(30)

,norm_ball_control,norm_crossing,norm_dribbling,norm_finishing,norm_long_passing,norm_overall_rating,norm_short_passing,norm_volleys,posdecile,position
0,0.494947,0.458953,0.582579,0.717209,0.376383,0.748297,0.471941,0.581046,1.0,FWD
1,0.535912,0.530601,0.507707,0.734404,0.543507,0.542409,0.536573,0.579865,2.0,FWD
2,0.551944,0.513080,0.567404,0.712828,0.483439,0.548217,0.598013,0.603800,3.0,FWD
3,0.451477,0.501680,0.615809,0.874158,0.529521,0.518289,0.569851,0.717939,4.0,FWD
4,0.741153,0.595326,0.719103,0.821615,0.456126,0.555418,0.632045,0.650484,5.0,FWD
5,0.613425,0.528004,0.590108,0.710494,0.493284,0.465451,0.427301,0.694399,6.0,FWD
6,0.585018,0.582905,0.569503,0.794303,0.574416,0.495170,0.589445,0.600853,7.0,FWD
7,0.399785,0.546718,0.409545,0.624028,0.541786,0.448401,0.511998,0.526876,8.0,FWD
8,0.489523,0.619371,0.568916,0.665114,0.506904,0.433584,0.575639,0.505628,9.0,FWD
9,0.431774,0.561016,0.542800,0.597272,0.442005,0.274676,0.389128,0.479812,10.0,FWD


In [35]:
JExport('avgposspecific.json',dfPosPlyrdecile);